# User based collaborative filtering
## Abstract
User based collaborative filtering is one of the algorithms located in the memory base method, calculates the similarity of users based on "being interested in the same item", and "similar user's evaluation is similar "Based on the similarity of the user, the recommendation is made.
The algorithm is shown below.

$$
\begin{equation}
    \hat{\gamma}_{uy} = \hat{\gamma_u} + \frac{
        \Sigma_{x\in{X_y}}  S_{ux} (\gamma_{xy} - \bar{\gamma_x}^{\prime})
    }{
        \Sigma_{x\in{X_y}} |S_{ux}|
    }
\end{equation}
$$

where:

$\hat{\gamma}_{uy}$ : Prediction that User $u$ evaluate Item $y$  
$X_{y}$ : Set of users who evaluating Item $y$  
$\bar{\gamma_u}$ : Average evaluation value for all evaluated items of user $u$  
$S_ux$ :Similarity between User $u$ and User $x$  
$\gamma_xy$ : Evaluation from User $x\in{X}$ to Item $y\in{Y}$  
$\bar{\gamma_x}^{\prime}$ : $\Sigma_{y\in{Y_{ux}}} \frac{\gamma_{xy}}{|\gamma_{ux}|}$


## How to calc similarity?
In case of IBCF, `Pearson correlation coefficient` is often used.


## Let's implement!
### 1st. Pre-processing
I used data [Sushi](http://www.kamishima.net/sushi/) .
Firstly, transform data to matrix.

In [36]:
import os
print(os.getcwd()) # if it show the path like '....../tech_note_for_machine_learning/data/sushi', it is ok without below
os.chdir('../../data/sushi')

/Users/maki/dev/src/github.com/simula-labs/tech_note_for_machine_learning/data/sushi


In [28]:
import numpy as np
import pandas as pd
from scipy import sparse

scores = np.loadtxt('sushi3b.5000.10.score', delimiter=' ')
scores

array([[-1.,  0., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1.,  3.,  4., ..., -1., -1., -1.],
       ...,
       [ 3., -1., -1., ..., -1., -1., -1.],
       [-1.,  4.,  4., ..., -1., -1., -1.],
       [-1., -1.,  3., ..., -1., -1., -1.]])

### 2nd. Calc corrcoef

In [29]:
# Sorry I have no time to inplement module, I wrote just functions
def get_correlation_coefficents(scores, target_user_index):
    similarities = []
    target = scores[target_user_index]
    
    for i, score in enumerate(scores):
        # 共通の評価が少ない場合は除外
        indices = np.where(((target + 1) * (score + 1)) != 0)[0]
        if len(indices) < 3 or i == target_user_index:
            continue
        
        similarity = np.corrcoef(target[indices], score[indices])[0, 1]
        if np.isnan(similarity):
            continue
    
        similarities.append((i, similarity))
    
    return sorted(similarities, key=lambda s: s[1], reverse=True)


target_user_index = 0 # 0番目のユーザ
similarities = get_correlation_coefficents(scores, target_user_index)
similarities

/Users/maki/.pyenv/versions/anaconda3-4.3.0/envs/dev/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/maki/.pyenv/versions/anaconda3-4.3.0/envs/dev/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


[(186, 1.0),
 (269, 1.0),
 (381, 1.0),
 (1045, 1.0),
 (1150, 1.0),
 (2411, 1.0),
 (2890, 1.0),
 (3590, 1.0),
 (4157, 1.0),
 (4385, 1.0),
 (4537, 1.0),
 (4732, 1.0),
 (4921, 1.0),
 (534, 0.9999999999999998),
 (3532, 0.9999999999999998),
 (4910, 0.9999999999999998),
 (4646, 0.9958705948858223),
 (2540, 0.9819805060619659),
 (4292, 0.9819805060619659),
 (2321, 0.9819805060619656),
 (2644, 0.9819805060619656),
 (2369, 0.9707253433941511),
 (262, 0.970725343394151),
 (1619, 0.970725343394151),
 (3258, 0.970725343394151),
 (847, 0.9707253433941508),
 (1383, 0.9707253433941508),
 (2067, 0.9707253433941508),
 (2166, 0.9707253433941508),
 (2648, 0.9707253433941508),
 (3411, 0.9707253433941506),
 (2085, 0.9684959969581862),
 (4287, 0.9655810287305759),
 (3553, 0.9607689228305228),
 (4237, 0.9607689228305228),
 (2553, 0.9607689228305226),
 (3527, 0.9607689228305226),
 (1003, 0.9449111825230683),
 (4345, 0.9449111825230683),
 (122, 0.944911182523068),
 (1634, 0.944911182523068),
 (3184, 0.94491118

### 3rd. Predict

In [33]:
def predict(scores, similarities, x, y):
    target = scores[x]
    
    avg_target = np.mean(target[np.where(target >= 0)])
    
    numerator = 0.0
    denominator = 0.0
    k = 0
    
    for similarity in similarities:
        # トップ５の類似度使う
        if k > 5 or similarity[1] <= 0.0:
            break
            
        score = scores[similarity[0]]
        if score[y] >= 0:
            denominator += similarity[1]
            numerator += similarity[1] * (score[y] - np.mean(score[np.where(score >= 0)]))
            k += 1
            
    return avg_target + (numerator / denominator) if denominator > 0 else -1

target_item_index = 0 # 3番目のアイテム(エビ)
predict(scores, similarities, target_user_index, target_item_index)

2.7609467700985615

In [34]:
def rank_items(scores, similarities, target_user_index):
    rankings = []
    target = scores[target_user_index]
    # 寿司ネタ100種類の全てで評価値を予測
    for i in range(100):
        # 既に評価済みの場合はスキップ
        if target[i] >= 0:
            continue

        rankings.append((i, predict(scores, similarities, target_user_index, i)))
        
    return sorted(rankings, key=lambda r: r[1], reverse=True)


target_user_index = 0 # 0番目のユーザ

rank = rank_items(scores, similarities, target_user_index)

In [35]:
rank

[(92, 2.85),
 (61, 2.7935924227841857),
 (0, 2.7609467700985615),
 (37, 2.7211344002240043),
 (22, 2.60003209164618),
 (84, 2.6),
 (5, 2.5946657805828632),
 (75, 2.555719199760294),
 (31, 2.4654723878573956),
 (53, 2.3676371202392996),
 (10, 2.3660238723670055),
 (38, 2.3650101398862566),
 (91, 2.3000000000000003),
 (40, 2.295560296444147),
 (19, 2.2760501211322266),
 (79, 2.2208255433546236),
 (9, 2.2146082646298866),
 (13, 2.1573704763234187),
 (70, 2.1485331036951845),
 (76, 2.1195194798790324),
 (2, 2.1),
 (47, 2.0565551731059193),
 (8, 1.9890817217052343),
 (78, 1.9785348971392018),
 (65, 1.9479070809100525),
 (82, 1.940689331738804),
 (88, 1.8867114232725926),
 (15, 1.846798191540819),
 (43, 1.7716437568938366),
 (26, 1.7645254789772584),
 (73, 1.7516268967471267),
 (34, 1.7431005911953052),
 (57, 1.7371937923981555),
 (17, 1.7172899268716972),
 (20, 1.655635066414566),
 (7, 1.6490878530385313),
 (51, 1.63820030388256),
 (30, 1.605215989030246),
 (42, 1.5787365846926897),
 (21, 1